In [1]:
from transformers import BertTokenizer, BertModel
import torch

# BERTのトークナイザーとモデルをロード
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese')

# 文字「ア」をトークン化
inputs = tokenizer("ア", return_tensors="pt")

# モデルを通して埋め込みを取得
with torch.no_grad():
    outputs = model(**inputs)

# 出力の中の埋め込み（最後の隠れ層の出力）
embedding = outputs.last_hidden_state
print(embedding)

/home/mitani/.conda/envs/sotuken/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-06 01:39:56.955017: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-06 01:39:58.103365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 01:39:58.437150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 01:39:58

tensor([[[-0.0856,  0.3027,  0.3669,  ...,  0.0771,  0.3301, -0.1210],
         [ 0.3103,  0.9518,  0.1079,  ...,  0.2146,  0.1853, -0.5152],
         [ 0.2058,  0.5051,  0.4948,  ...,  0.1341, -0.0217, -0.0406]]])


In [ ]:
# 768次元
# BERTで文字のエンベディングを取得
# 保存先；embeddings_output
import os
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# トークナイザーとモデルのロード
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese')

# 入力ディレクトリと出力ディレクトリ
input_dir = 'basic5000_HuBERT_features_csv'     # HuBERTで抽出した特徴量F
output_dir = 'embeddings_output'

# 出力ディレクトリを作成
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 任意の個数を指定してファイルを読み込む関数
def load_files(num_files=None):
    # ディレクトリ内のCSVファイルを取得
    files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]
    
    # ソート（昇順）して最初の num_files 個を選択
    files.sort()  # ファイル名をソートして順番を決定
    if num_files is None or num_files > len(files):
        selected_files = files
    else:
        selected_files = files[:num_files]
    
    return selected_files

# 任意の個数のファイルを選択
num_files_to_process = 5000  # ここで読み込むファイル数を指定（例: 5）
selected_files = load_files(num_files=num_files_to_process)

# CSVファイルを順番に処理
for csv_file in selected_files:
    # CSVファイルを読み込む
    file_path = os.path.join(input_dir, csv_file)
    df = pd.read_csv(file_path)

    # Mora列を取り出し、エンベディングを計算
    moras = df['Mora'].tolist()
    
    # エンベディング結果を保存するリスト
    embeddings = []

    for mora in moras:
        # Moraをトークン化し、エンベディングを計算
        inputs = tokenizer(mora, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        # BERTの出力の[CLS]トークンのベクトル（通常最初のトークン）を使用
        embedding = outputs.last_hidden_state[0][0].detach().numpy()
        embeddings.append(embedding)

    # 新しいデータフレームを作成
    embedding_df = pd.DataFrame({
        'Mora': moras,
        'Embedding': embeddings
    })

    # 出力ファイル名を設定
    output_file = os.path.join(output_dir, f'embeddings_{csv_file}')
    
    # エンベディングをCSVファイルとして保存
    embedding_df.to_csv(output_file, index=False)

    print(f"Processed {csv_file}, saved to {output_file}")

/home/mitani/.conda/envs/sotuken/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-09 16:31:24.689529: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 16:31:24.696826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 16:31:24.705650: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 16:31:24

FileNotFoundError: [Errno 2] No such file or directory: 'basic5000_HuBERT_features_csv'

In [ ]:
# 不必要
# 768次元必要なところ、767次元しかない
# HuBERTの特徴量の次元数が足りない所をゼロパディング
import os
import pandas as pd
import numpy as np

# 入力ディレクトリと出力ディレクトリ
input_dir = "basic5000_features_with_accent"
output_dir = "padded_features"
os.makedirs(output_dir, exist_ok=True)

# ゼロパディングを用いて768次元に揃える関数
def zero_pad_to_768(features, target_dim=768):
    current_dim = len(features)
    if current_dim < target_dim:
        return np.pad(features, (0, target_dim - current_dim), mode="constant")  # ゼロでパディング
    return features[:target_dim]  # 768次元に超えている場合、切り捨て

# 指定された数のファイルを処理
def process_files(input_dir, output_dir, num_files):
    files = sorted([f for f in os.listdir(input_dir) if f.endswith(".csv")])  # ソートして順序を固定
    selected_files = files[:num_files]  # 上からnum_files個のファイルを選択

    for file in selected_files:
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, file)

        # ファイルの読み込み
        df = pd.read_csv(input_path)

        # Mora列とアクセント列を取得
        mora_column = df["Mora"]
        accent_column = df["Accent"]

        # 音響特徴量列（Feature_0 から Feature_767）を抽出
        feature_columns = [f"Feature_{i}" for i in range(768)]  # Feature_0 から Feature_767, 768→500
        features = df[feature_columns].values

        # 各行の音響特徴量をゼロパディングで補完
        padded_features = []
        for row in features:
            # 非NaN値を取り出してゼロパディング
            row_no_nan = row[~np.isnan(row)]  # NaNを取り除く
            padded_row = zero_pad_to_768(row_no_nan)  # ゼロパディング
            padded_features.append(padded_row)

        # データフレームに再構成
        padded_df = pd.DataFrame(
            padded_features, columns=feature_columns
        )
        padded_df.insert(0, "Mora", mora_column)  # Mora列を追加
        padded_df["Accent"] = accent_column  # Accent列を追加

        # CSVとして保存
        padded_df.to_csv(output_path, index=False)

# 実行例: 入力ディレクトリ内の上から5ファイルを処理
num_files_to_process = 5  # 任意のファイル数
process_files(input_dir, output_dir, num_files_to_process)

In [ ]:
# 768次元だった
# パディング出来てるかを確認するコード
# 何次元かをカウント可能

import os
import pandas as pd

# 入力ディレクトリ（padded_features）と出力ディレクトリ（output_dir）のパス
# input_dir = "sample_HuBERT_features_csv"
# output_dir = "sample_HuBERT_features_csv"
# input_dir = "padded_features"
input_dir = "basic5000_features_with_accent"
output_dir = "feature_counts_output"  # 出力先ディレクトリ

# 出力ディレクトリが存在しない場合は作成
os.makedirs(output_dir, exist_ok=True)

# 任意のファイル数を指定（例: 最初の3ファイルを処理）
num_files_to_process = 5  # ここで処理するファイルの数を指定します

# 入力ディレクトリ内の全てのCSVファイルを取得
all_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# 処理するファイルをリストの先頭から指定された数だけ取得
file_list = all_files[:num_files_to_process]

# 各ファイルを処理
for file_name in file_list:
    # ファイルパスを作成
    input_file = os.path.join(input_dir, file_name)

    # ファイルが存在する場合のみ処理
    if os.path.exists(input_file):
        # CSVファイルを読み込み（ヘッダーを無視して2行目以降を読み込む）
        df = pd.read_csv(input_file, header=0)

        # 特徴量列の開始位置を特定 (Feature_0 ～ Feature_N)
        feature_columns = [col for col in df.columns if col.startswith("Feature_")]

        # 各音節の特徴量の数をカウントする
        df_counts = df[["Mora"]].copy()  # Mora列をコピー
        df_counts["Feature_Count"] = df[feature_columns].notnull().sum(axis=1)  # 非NaNの特徴量の数をカウント

        # 出力ファイルのパスを設定（ファイル名に対応した名前を付ける）
        output_file = os.path.join(output_dir, f"{file_name}_feature_counts.csv")

        # 結果を新しいCSVファイルに保存
        df_counts.to_csv(output_file, index=False)

        print(f"{file_name} の特徴量のカウント結果を {output_file} に保存しました。")
    else:
        print(f"{file_name} が見つかりません。")

alignment_1559.csv の特徴量のカウント結果を feature_counts_output/alignment_1559.csv_feature_counts.csv に保存しました。
alignment_0341.csv の特徴量のカウント結果を feature_counts_output/alignment_0341.csv_feature_counts.csv に保存しました。
alignment_0633.csv の特徴量のカウント結果を feature_counts_output/alignment_0633.csv_feature_counts.csv に保存しました。
alignment_4265.csv の特徴量のカウント結果を feature_counts_output/alignment_4265.csv_feature_counts.csv に保存しました。
alignment_2959.csv の特徴量のカウント結果を feature_counts_output/alignment_2959.csv_feature_counts.csv に保存しました。


In [ ]:
# embedding_outputがない！, features_karam生成
# 文字数が合わない！
# Mora,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,...,Feature_767と横一列にした
import pandas as pd
import numpy as np
import os

# 特徴量を展開してCSVに保存する関数
def convert_embeddings_to_features(input_dir, output_dir, num_files=None):
    # 入力ディレクトリ内のすべてのファイルを取得
    files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]
    
    if num_files is not None:
        files = files[:num_files]  # 最初のnum_files個のファイルだけを処理
    
    for file in files:
        file_path = os.path.join(input_dir, file)
        df = pd.read_csv(file_path)
        
        # 新しいデータフレームを作成
        features = []

        # 音節（Mora）とその特徴量を変換
        for _, row in df.iterrows():
            mora = row['Mora']
            embedding_str = row['Embedding']
            # numpy配列として変換
            embedding = np.fromstring(embedding_str.strip('[]'), sep=' ', dtype=np.float32)

            # 音節とその特徴量を新しい行として保存
            features.append([mora] + embedding.tolist())

        # 新しいデータフレームを作成
        feature_columns = ['Mora'] + [f'MFeature_{i}' for i in range(embedding.shape[0])]
        feature_df = pd.DataFrame(features, columns=feature_columns)

        # 出力ファイルパス
        output_file_path = os.path.join(output_dir, file)
        feature_df.to_csv(output_file_path, index=False)
        print(f"Saved: {output_file_path}")

# 実行部分
input_directory = "./embeddings_output"  # 特徴量が保存されているディレクトリ
output_directory = "./processed_embeddings_karam"  # 処理後の特徴量を保存するディレクトリ
os.makedirs(output_directory, exist_ok=True)

num_files_to_process = 3  # 任意のファイル数（例: 最初の5ファイルを処理）

# 特徴量を変換して保存
convert_embeddings_to_features(input_directory, output_directory, num_files=num_files_to_process)

FileNotFoundError: [Errno 2] No such file or directory: './embeddings_output'